#### Ciencia de datos - Primavera 2024
##### Integrantes: Magdalena Cobb, Pedro García Vassallo, Marcos Olavarría
### Trabajo Práctico 4

In [1]:
#Paquetes y librerías a usar

#!pip install openpyxl
#!pip install statsmodels

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm 

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
#from sklearn.metrics import plot_roc_curve
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

In [2]:
#Lectura del documento
ind_2024 = pd.read_excel('C:/Users/magda/OneDrive/Documents/GitHub/CC408-T1-5/TP 4/usu_individual_T124.xlsx') 
ind_2004 = pd.read_stata('C:/Users/magda/OneDrive/Documents/GitHub/CC408-T1-5/TP 4/Individual_t104.dta')
hog_2024 = pd.read_excel('C:/Users/magda/OneDrive/Documents/GitHub/CC408-T1-5/TP 4/usu_hogar_T124.xlsx')
hog_2004 = pd.read_stata('C:/Users/magda/OneDrive/Documents/GitHub/CC408-T1-5/TP 4/Individual_t104.dta')

#Maggie: C:/Users/magda/OneDrive/Documents/GitHub/CC408-T1-5/TP 4/
#Peter: 'C:/Users/SAMSUNG/Documents/GitHub/CC408-T1-5/TP 3/Individual_t104.dta


a. Eliminen todas las observaciones que no corresponden a los
aglomerados de Ciudad Autónoma de Buenos Aires o Gran Buenos
Aires, y unan ambos trimestres en una sola base.

In [ ]:
#Filtramos las bases para tener solo Buenos Aires
df_2024_GB = df2024[(df2024['AGLOMERADO'] == 33) | (df2024['AGLOMERADO'] == 32)]
df_2004_GB = df2004[(df2004['aglomerado'] == 'Ciudad de Buenos Aires') | (df2004 ['aglomerado'] == 'Partidos del GBA' )]

df_2004_GB.columns = df_2004_GB.columns.str.upper()
df_2024_GB['IPCF'] =df_2024_GB['IPCF'].astype(int)

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_13684\2033928064.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2024_GB['IPCF'] =df_2024_GB['IPCF'].astype(int)


#### Se eliminan columnas antes de combinar bases

In [ ]:
#IDENTIFICO COLUMNAS QUE DIFIEREN ENTRE LAS BASES
col2004 = df_2004_GB.columns
col2024 = df_2024_GB.columns

# Encontrar columnas que son diferentes
columnas_df1 = set(col2004)
columnas_df2 = set(col2024)

# Columnas en df1 pero no en df2
diferentes_df1 = columnas_df1 - columnas_df2
# Columnas en df2 pero no en df1
diferentes_df2 = columnas_df2 - columnas_df1

# Comparar columnas comunes para ver diferencias en valores
diferencias_valores = {}
for col in columnas_df1.intersection(columnas_df2):
    if not df_2004_GB[col].equals(df_2024_GB[col]):
        diferencias_valores[col] = (df_2004_GB[col].tolist(), df_2024_GB[col].tolist())

print("Columnas solo en 2004:", diferentes_df1)
print("Columnas solo en 2024:", diferentes_df2)
#print("Diferencias en valores:", diferencias_valores)


Columnas solo en 2004: {'PJ1_1', 'IDIMPP', 'PJ2_1', 'PJ3_1'}
Columnas solo en 2024: {'PONDIIO', 'CH05', 'PONDII', 'PONDIH', 'IMPUTA'}


In [ ]:
#ELIMINO LAS COLUMNAS QUE DIFIEREN Y LAS QUE RESULTAN IRRELEVANTES
del_col1 = ['PJ3_1', 'IDIMPP', 'PJ2_1', 'PJ1_1', 'CODUSU']
del_col2 = ['IMPUTA', 'CH05', 'PONDII', 'PONDIH', 'PONDIIO','CODUSU']

df_2004_GB =df_2004_GB.drop(columns = del_col1)
df_2024_GB =df_2024_GB.drop(columns = del_col2)

# df_combined = df.drop(columns=['B', 'C'])

#### Se combinan las bases
b. Si hay observaciones con valores que no tienen sentido,
descártenlas (por ejemplo, ingresos y edades negativos). Expliquen
las decisiones tomadas.

In [ ]:
# Uno ambos trimestres en una sola base
df_combined = pd.concat([df_2004_GB, df_2024_GB], ignore_index=True, sort=False)
df_combined["ANO4"] = df_combined["ANO4"].astype(int)


In [ ]:
#Definición de algunas variables para uniformar diccionarios y unificar las bases
h04 = {
    "Varón": 1,
    "Mujer": 2
}

h06 = {
    'Menos de 1 año': 0,
    '98 y más años': 98
}

h07 = {
    "Unido": 1,
    "Casado": 2,
    "Separado o divorciado": 3,
    "Viudo": 4,
    "Soltero": 5
}

h08 =    {
    'Obra social (incluye PAMI)': 1,
    'Mutual/Prepaga/Servicio de emergencia': 2,
    'Planes y seguros públicos': 3,
    'No paga ni le descuentan': 4,
    'Ns./Nr.': 9,
    'Obra social y mutual/prepaga/servicio de emergencia': 12,
    'Obra social y planes y seguros públicos': 13,
    'Mutual/prepaga/servicio de emergencia/planes y seguros públi': 23,
    'Obra social, mutual/prepaga/servicio de emergencia y planes': 123
}

nivel_ed= {
    'Primaria Incompleta (incluye educación especial)': 1,
    'Primaria Completa': 2,
    'Secundaria Incompleta': 3,
    'Secundaria Completa': 4,
    'Superior Universitaria Incompleta': 5,
    'Superior Universitaria Completa': 6,
    'Sin instrucción': 7,
    'Ns/Nr': 9
}

estado = {
    'Entrevista individual no realizada (no respuesta al cuestion': 0,
    'Ocupado': 1,
    'Desocupado': 2,
    'Inactivo': 3,
    'Menor de 10 años': 4
}

cat_inac = {
    'Jubilado/pensionado': 1,
    'Rentista': 2,
    'Estudiante': 3,
    'Ama de casa': 4,
    'Menor de 6 años': 5,
    'Discapacitado': 6,
    'Otros': 7
}


componente = {
    'Servicio doméstico en hogares': 51
}

h15 = {
    "Sí": 1,
    "No": 2
}

mas_500 = {
    'N' : 1,
    'S' : 0
} 

aglomerado = {
    'Ciudad de Buenos Aires' : 32,
    'Partidos del GBA' : 33

}

ch03 = {
    'Jefe' : 1,
    'Cónyuge/Pareja' : 2,
    'Hijo/Hijastro' : 3,
    'Yerno/Nuera' : 4,
    'Nieto' : 5,
    'Madre/Padre' : 6,
    'Suegro' : 7,
    'Hermano' : 8,
    'Otros familiares' : 9,
    'No familiares' : 10,
}

ch09 = {
    "Sí": 1,
    "No": 2,
    "Menor de 2 años": 3,
    'Ns./Nr.' : 0 
}

ch10 = {
    "Sí, asiste": 1,
    "No asiste, pero asistió": 2,
    "Nunca asistió": 3  
}

In [ ]:
df_combined.loc[:,'CAT_INAC'] = df_combined['CAT_INAC'].replace(cat_inac).astype(int) 
df_combined.loc[:,'CH04'] = df_combined['CH04'].replace(h04).astype(int)
df_combined.loc[:,'CH06'] = df_combined['CH06'].replace(h06).astype(int)
df_combined.loc[:,'CH07'] = df_combined['CH07'].replace(h07).astype(int)
df_combined.loc[:,'CH08'] = df_combined['CH08'].replace(h08).astype(int)
#df_combined.loc[:,'AGLOMERADO'] = df_combined['AGLOMERADO'].map(aglomerado).astype(int)
df_combined.loc[:,'NIVEL_ED'] = df_combined['NIVEL_ED'].replace(nivel_ed).astype(int)
df_combined.loc[:,'ESTADO'] = df_combined['ESTADO'].replace(estado).astype(int)
df_combined['IPCF'] = df_combined['IPCF'].astype(int)

df_combined.loc[:,'COMPONENTE'] = df_combined['COMPONENTE'].replace(componente).astype(int)
df_combined.loc[:,'H15'] = df_combined['H15'].replace(h15).astype(int)
df_combined.loc[:,'MAS_500'] = df_combined['MAS_500'].replace(mas_500).astype(int)
df_combined.loc[:,'AGLOMERADO'] = df_combined['AGLOMERADO'].replace(aglomerado).astype(int)
df_combined.loc[:, 'CH03'] = df_combined['CH03'].replace(ch03).astype(int)
df_combined.loc[:, 'CH09'] = df_combined['CH09'].replace(ch09).astype(int)  # hay 2 valores 9 que no deberían estar
df_combined.loc[:, 'CH10'] = df_combined['CH10'].replace(ch10).astype(int)  # hay 364 valores que son 0 y 2 que son 9 que no deberían ser

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_13684\3711807908.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined.loc[:,'CAT_INAC'] = df_combined['CAT_INAC'].replace(cat_inac).astype(int)
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_13684\3711807908.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined.loc[:,'CH04'] = df_combined['CH04'].replace(h04).astype(int)
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_13684\3711807908.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be rem

In [ ]:
valores_validos = {
    'CH04': [1,2],
    'CH07': [1,2,3,4,5],
    'CH08': [1,2,3,4,9,12,13,23,123],
    'NIVEL_ED': [1,2,3,4,5,6,7,8,9],
    'ESTADO': [0,1,2,3,4],
    'CAT_INAC': [0,1,2,3,4,5,6,7] 
} #'CH06' Y 'IPCF' quedan excluidos porque tiene condiciones de min y max

for col in valores_validos.keys():
    df_combined = df_combined[df_combined[col].isin(valores_validos[col])]

In [ ]:
# Define los límites
maximo_edad = 130 # totalmente arbitrario, pero sirve para sacar extraños muy extraños

# Filtra el DataFrame, manteniendo solo los valores dentro del rango
df_combined = df_combined[(df_combined['IPCF'] >= 0)]
df_combined = df_combined[(df_combined['CH06'] >= 0) & (df_combined['CH06'] <= maximo_edad)]

In [ ]:
df_combined

,NRO_HOGAR,COMPONENTE,H15,ANO4,TRIMESTRE,REGION,MAS_500,AGLOMERADO,PONDERA,CH03,...,GDECIFR,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR
0,1.0,1.0,Sí,2004,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1299.0,Jefe,...,02,,01,368,07,,06,06,,04
1,1.0,1.0,Sí,2004,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,2018.0,Jefe,...,00,,00,0,00,,00,00,,00
2,1.0,1.0,Sí,2004,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1486.0,Jefe,...,03,,02,440,07,,07,07,,05
3,1.0,1.0,Sí,2004,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1674.0,Jefe,...,09,,08,532,08,,08,08,,06
4,1.0,2.0,Sí,2004,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1674.0,Cónyuge/Pareja,...,09,,08,532,08,,08,08,,06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14693,1.0,1,1,2024,1,1,S,33,3410.0,1,...,12.0,NaN,12,0,12,NaN,12,12.0,NaN,12
14694,1.0,2,2,2024,1,1,S,33,3410.0,3,...,12.0,NaN,12,0,12,NaN,12,12.0,NaN,12
14695,1.0,3,2,2024,1,1,S,33,3410.0,3,...,12.0,NaN,12,0,12,NaN,12,12.0,NaN,12
14696,1.0,4,2,2024,1,1,S,33,3410.0,3,...,12.0,NaN,12,0,12,NaN,12,12.0,NaN,12
